# <h1 style="text-align: center"><b>184.702 Machine Learning</b>: Exercise 0, 2022S</h1>
<h5 style="text-align: center">Group 08: Wanecek Wilhelm, Simhandl Stefan, Beck Viktor</h5>

In [ ]:
# Install dependencies
!pip3 install -q pandas_profiling

In [ ]:
import os
path= ''
currentdirectory = os.path.abspath(path)
os.chdir(currentdirectory)

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# Kinematics-motion-dataset

source:
https://www.kaggle.com/yasserh/kinematics-motion-data

In [ ]:
# import raw data

csvpath = os.path.join("datasets", "kinematics-motion-data.csv")
df=pd.read_csv(csvpath, sep=',', encoding='latin-1')
df.info()
df.head()

## Explore kinematics-motion-dataset

In [ ]:
df.shape
df.isna().sum()

In [ ]:
# get descriptive statistics
df.describe()

In [ ]:
# explor dataset using pandas profiling
from pandas_profiling import ProfileReport

# 1) Date, Time and Username are no universal features to predict the activites Running and Walking 
# 2) Further it makes no sense to explore them so we can drop them before exploring the data
df = df.drop(columns=['username','date','time'],axis=1)

prof = ProfileReport(df)#, minimal = True)
prof.to_file(output_file='explore_kinematics-motion-dataset.html')

# Communities and Crime dataset

source:
https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime

In [ ]:
# import raw data
csvpath = os.path.join("datasets", "communities_crime.csv")
df=pd.read_csv(csvpath, sep=';', encoding='latin-1')

df.info()
df.head()

## Explore communities and crime dataset

In [ ]:
df.shape
df.isna().sum()

In [ ]:
# get descriptive statistics
df.describe()

In [ ]:
# explor dataset using pandas profiling
from pandas_profiling import ProfileReport

prof = ProfileReport(df, minimal = True)
prof.to_file(output_file='explore_communities-crime-dataset.html')